In [20]:
import numpy as np
import xgboost as xgb
from collections import deque
from Dynaforest import Dynatree
from sklearn.utils.estimator_checks import check_estimator
from sklearn.datasets import make_regression
from sklearn.utils.validation import check_X_y, check_array


def create_regression_data(n_points, n_features):
    X = np.random.rand(n_points, n_features)
    y = X[:, 0] * 5 + np.sign(X[:, 1]) + np.sin(X[:, 2]) + 1 + np.random.randn(n_points)
    return X, y


X, y = make_regression(
    n_samples = 200000,
    n_features = 10, 
    n_informative=5, 
    noise=50,
    bias = 5.0, 
    random_state=42
)

model = xgb.XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1)
#model = Dynatree(n_trees=5, window=2, max_depth=3, min_samples=3)
model.fit(X, y)
print(model.score(X, y))  # Should be near 1.0 if it splits properly



0.9167636087505203


In [6]:
a = np.array([
    [1, 2, 3, 4, 5, 5], 
    [1, 2, 3, 4, 5, 6]]
)
a[1, :]

array([1, 2, 3, 4, 5, 6])

In [ ]:
from scan_thresholds import get_best_sse3, get_best_sse
test_X = np.arange(0, 5, 0.2).reshape(-1, 1)
test_y = np.sin(test_X) - np.cos(test_X) 
test_y = test_y.ravel()

forest = Dynatree(n_trees=10, window = 4)
forest.fit(test_X, test_y)
print(forest._trees[0].root)
print(test_y - forest.predict(test_X))
print(np.sum((forest.predict(test_X) - test_y)**2))


In [12]:
sort_idx = np.argsort(test_X[:, 1])
X_col_sorted = test_X[:, 1][sort_idx]
y_sorted = test_y[sort_idx]
other_predictions_sorted = test_other_predictions[sort_idx]

alpha = 3

prefix_sum_y = np.cumsum(y_sorted)
prefix_sum_y_squared = np.cumsum(y_sorted**2)
prefix_sum_other_predictions = np.cumsum(other_predictions_sorted)
prefix_sum_other_predictions_squared = np.cumsum(other_predictions_sorted**2)
prefix_sum_multiplication = np.cumsum(y_sorted * other_predictions_sorted)
prefix_idxs = list(range(1, len(y_sorted) + 1))

left_error = (prefix_sum_y_squared -
            (2*alpha) / (alpha + 1) * prefix_sum_multiplication -
            2 / (alpha + 1) * prefix_sum_y**2 / prefix_idxs + 
            prefix_sum_other_predictions_squared * (alpha / (alpha + 1))**2 + 
                (2*alpha) / (alpha + 1)**2 * prefix_sum_y * prefix_sum_other_predictions / prefix_idxs + 
                prefix_sum_y**2 / prefix_idxs * 1/(alpha + 1)**2)

suffix_sum_y = np.sum(y_sorted) - prefix_sum_y
suffix_sum_y_squared = np.sum(y_sorted**2) - prefix_sum_y_squared
suffix_sum_other_predictions = np.sum(other_predictions_sorted) - prefix_sum_other_predictions
suffix_sum_other_predictions_squared = np.sum(other_predictions_sorted**2) - prefix_sum_other_predictions_squared
suffix_sum_multiplication = np.sum(y_sorted * other_predictions_sorted) - prefix_sum_multiplication
suffix_idxs = list(range(len(y_sorted) - 1, -1, -1))

right_error = (suffix_sum_y_squared -
                        (2*alpha) / (alpha + 1) * suffix_sum_multiplication -
                        2 / (alpha + 1) * suffix_sum_y**2 / suffix_idxs +
                        suffix_sum_other_predictions_squared * (alpha / (alpha + 1))**2 +
                        (2*alpha) / (alpha + 1)**2 * suffix_sum_y * suffix_sum_other_predictions / suffix_idxs +
                        suffix_sum_y**2 / suffix_idxs * 1/(alpha + 1)**2)

sum_error = left_error + right_error
print(sum_error)
print(np.argmin(sum_error))



[0.90875 0.45875     nan]
2


/var/folders/b1/dp_lgrnx2s71rj9w2dv4fhbm0000gn/T/ipykernel_88704/1756357109.py:31: RuntimeWarning: invalid value encountered in divide
  2 / (alpha + 1) * suffix_sum_y**2 / suffix_idxs +
/var/folders/b1/dp_lgrnx2s71rj9w2dv4fhbm0000gn/T/ipykernel_88704/1756357109.py:33: RuntimeWarning: invalid value encountered in divide
  (2*alpha) / (alpha + 1)**2 * suffix_sum_y * suffix_sum_other_predictions / suffix_idxs +
/var/folders/b1/dp_lgrnx2s71rj9w2dv4fhbm0000gn/T/ipykernel_88704/1756357109.py:34: RuntimeWarning: invalid value encountered in divide
  suffix_sum_y**2 / suffix_idxs * 1/(alpha + 1)**2)


In [7]:
X = np.array([1, 2, 3, 4, 5]).reshape(-1, 1)
y = [2.5, 0, -5, 10, 7.5]
forest = Dynatree(n_trees = 10, window = 4, max_depth = 3, min_samples = 1)
forest.fit(X, y)

#Test requirements for splitting
print(forest.trees[0].root.curr_best_splitting_val)
assert forest.trees[0].root.left.curr_best_splitting_val == 2


Splitting tree, error reduction: 155.20833333333334
Creating new tree, error reduction: 82.65625
Splitting tree, error reduction: 19.53125
Creating new tree, error reduction: 15.306712962962933
Splitting tree, error reduction: 14.467592592592592
Creating new tree, error reduction: 5.357349537037038
Splitting tree, error reduction: 11.393229166666668
Creating new tree, error reduction: 2.47968749999999
Splitting tree, error reduction: 11.393229166666671
Splitting tree, error reduction: 1.3671875
Splitting tree, error reduction: 1.3671875
Splitting tree, error reduction: 1.3671875
Splitting tree, error reduction: 1.3671875
Splitting tree, error reduction: 1.3671875
Splitting tree, error reduction: 1.3671875
Splitting tree, error reduction: 1.3671875
Splitting tree, error reduction: 1.3671875
No more error reduction possible
2


AssertionError: 

In [14]:
from scan_thresholds import get_best_sse3, get_best_sse

get_best_sse(
    X = np.array([1, 2, 3, 4, 5]).reshape(-1, 1), 
    y = np.array([2.5, 3, 5.5, 7, 8.5]),
    other_predictions=np.array([2.75, 2.75, 5.5, 7.75, 7.75]), 
    num_cols_other_predictions=1, 
    min_samples=1
)

(2.09375, 0, 2)

In [12]:
np.sum((np.array([2.5, 3, 5.5, 7, 8.5]) - np.array([2.75, 2.75, 5.5, 7.75, 7.75]))**2)

1.25